In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import geopandas as gpd
import re

#### Adults ER visits

In [2]:
#er visits
#adults, 2017-2019
er_18 = pd.read_csv('../Data/asthma_er_18_up_2017_2019.csv')
er_18.head(2)

,Time,GeoTypeDesc,GeoID,GeoRank,Geography,"Average annual age-adjusted rate per 10,000",Average annual number,"Average annual rate per 10,000"
0,2017-2019,Neighborhood Tabulation Area,501,8,Claremont-Bathgate,289.1,712,296.9
1,2017-2019,Neighborhood Tabulation Area,503,8,Eastchester-Edenwald-Baychester,158.4,453,158.4


In [3]:
er_18.dtypes

Time                                            object
GeoTypeDesc                                     object
GeoID                                            int64
GeoRank                                          int64
Geography                                       object
Average annual age-adjusted rate per 10,000    float64
Average annual number                           object
Average annual rate per 10,000                 float64
dtype: object

In [4]:
#convert avg annual number column to int
er_18['Average annual number'] = er_18['Average annual number'].apply(lambda x: re.sub('[^0-9]', '', x)).astype(int)

#### Children 5-17 ER visits

In [5]:
#er visits
#children 5-17, 2017-2019
er_5 = pd.read_csv('../Data/asthma_er_5_17_2017_2019.csv')
er_5.head(2)

,Time,GeoTypeDesc,GeoID,GeoRank,Geography,Average annual number,"Average annual rate per 10,000"
0,2017-2019,Neighborhood Tabulation Area,501,8,Claremont-Bathgate,298,399.5
1,2017-2019,Neighborhood Tabulation Area,503,8,Eastchester-Edenwald-Baychester,176,271.2


In [6]:
er_5.dtypes

Time                               object
GeoTypeDesc                        object
GeoID                               int64
GeoRank                             int64
Geography                          object
Average annual number               int64
Average annual rate per 10,000    float64
dtype: object

In [7]:
#convert avg annual number column to int
er_5['Average annual number'] = er_5['Average annual number'].astype(int)

#### Children 0-4 ER visits

There is no 2017-2019 asthma ER visits data for children aged 4 and under at NTA level. However, the citywide data is available. Therefore, we will need to assume every NTA followed the citywide trend and extrapolate the data this way. Since the NYCDOH website only reports the average annual rate per 10,000, we will multiply the 2014-2016 data by the ratio of the citywide $\frac{mean(2017-2019)}{mean(2012-2014)}$ to infer the NTA level data for this age group for 2017-2019.

In [8]:
#calculate means for the two periods
m_2014_2016 = np.mean([319.8, 307, 292.6])
m_2017_2019 = np.mean([253.7, 240.3, 194.7])
ratio = m_2017_2019/m_2014_2016
ratio

0.7490754840113119

In [9]:
er_0 = pd.read_csv('../Data/asthma_er_0_4_2014_2016.csv')
er_0.head(2)

,Time,GeoTypeDesc,GeoID,GeoRank,Geography,"Average annual rate per 10,000",Number
0,2014-2016,Neighborhood Tabulation Area,501,8,Claremont-Bathgate,801.6,762
1,2014-2016,Neighborhood Tabulation Area,503,8,Eastchester-Edenwald-Baychester,451.6,307


In [10]:
er_0.dtypes

Time                              object
GeoTypeDesc                       object
GeoID                              int64
GeoRank                            int64
Geography                         object
Average annual rate per 10,000    object
Number                            object
dtype: object

In [11]:
#convert the avg rate and number columns to float and int, respectively
er_0['Average annual rate per 10,000'] = er_0['Average annual rate per 10,000'].apply(lambda x: re.sub('[^0-9.]', '', x)).astype(float)
er_0['Average annual rate per 10,000'] = round(er_0['Average annual rate per 10,000']*ratio, 1)
er_0['Number'] = er_0['Number'].apply(lambda x: re.sub('[^0-9]', '', x)).astype(int)
er_0.head(2)

,Time,GeoTypeDesc,GeoID,GeoRank,Geography,"Average annual rate per 10,000",Number
0,2014-2016,Neighborhood Tabulation Area,501,8,Claremont-Bathgate,600.5,762
1,2014-2016,Neighborhood Tabulation Area,503,8,Eastchester-Edenwald-Baychester,338.3,307


#### Combine the three dataframes above

In [12]:
#first merge
#er_18 and er_5
m_1 = er_18.merge(er_5[['Geography', 'Average annual number', 'Average annual rate per 10,000']], 
                  how='inner', left_on='Geography', right_on='Geography')

#second merge
#m_1 and er_0
m_2 = m_1.merge(er_0[['Geography', 'Average annual rate per 10,000', 'Number']], 
                how='inner', left_on='Geography', right_on='Geography')
print(len(m_2))
m_2.head(2)

188


,Time,GeoTypeDesc,GeoID,GeoRank,Geography,"Average annual age-adjusted rate per 10,000",Average annual number_x,"Average annual rate per 10,000_x",Average annual number_y,"Average annual rate per 10,000_y","Average annual rate per 10,000",Number
0,2017-2019,Neighborhood Tabulation Area,501,8,Claremont-Bathgate,289.1,712,296.9,298,399.5,600.5,762
1,2017-2019,Neighborhood Tabulation Area,503,8,Eastchester-Edenwald-Baychester,158.4,453,158.4,176,271.2,338.3,307


In [13]:
#sum average annual number for all age groups
m_2['avg_num'] = m_2['Average annual number_x']+m_2['Average annual number_y']+m_2['Number']

#rename adult age-adjusted rate column
m_2 = m_2.rename(columns={'Average annual age-adjusted rate per 10,000':'aaa_rate', 
                          'Average annual rate per 10,000_x':'avg_rat_18',
                          'Average annual rate per 10,000_y':'avg_rat_5',
                          'Average annual rate per 10,000':'avg_rat_0'})
m_2 = m_2[['GeoID', 'GeoRank', 'Geography', 'aaa_rate', 'avg_rat_18', 'avg_rat_5', 'avg_rat_0', 'avg_num']]
m_2.head(2)

,GeoID,GeoRank,Geography,aaa_rate,avg_rat_18,avg_rat_5,avg_rat_0,avg_num
0,501,8,Claremont-Bathgate,289.1,296.9,399.5,600.5,1772
1,503,8,Eastchester-Edenwald-Baychester,158.4,158.4,271.2,338.3,936


The average rate for all age group combined needs to be calculated with the total population in each NTA.

In [14]:
m_2.to_csv('../Data/asthma_er_nta.csv', index=False)

#### Add the asthma data to NTA shapefile

In [15]:
#the data above is generated using 2010 NTA
nta = gpd.read_file('../Data/nta_2010/geo_export_d66993ca-6cc0-4ee0-9b66-052e7675b5f0.shp')
nta.head()

,boro_code,boro_name,county_fip,ntacode,ntaname,shape_area,shape_leng,geometry
0,4.0,Queens,081,QN08,St. Albans,7.741275e+07,45401.316803,"POLYGON ((-73.75205 40.70523, -73.75174 40.704..."
1,2.0,Bronx,005,BX28,Van Cortlandt Village,2.566612e+07,21945.719299,"POLYGON ((-73.88705 40.88435, -73.88705 40.884..."
2,4.0,Queens,081,QN55,South Ozone Park,8.246139e+07,36708.169305,"POLYGON ((-73.80577 40.68293, -73.80552 40.682..."
3,3.0,Brooklyn,047,BK40,Windsor Terrace,1.404167e+07,19033.672066,"POLYGON ((-73.98017 40.66115, -73.98021 40.661..."
4,3.0,Brooklyn,047,BK50,Canarsie,8.208968e+07,43703.609666,"MULTIPOLYGON (((-73.88834 40.64671, -73.88835 ..."


In [16]:
nta_df = pd.DataFrame(nta)

In [17]:
#check that nta names match 
print(len(set(nta_df['ntaname']).intersection(set(m_2['Geography']))))

188


In [18]:
#merge m_2 with nta
nta_m = nta_df.merge(m_2, how='inner', left_on='ntaname', right_on='Geography')
print(len(nta_m))
nta_m.head(2)

188


,boro_code,boro_name,county_fip,ntacode,ntaname,shape_area,shape_leng,geometry,GeoID,GeoRank,Geography,aaa_rate,avg_rat_18,avg_rat_5,avg_rat_0,avg_num
0,4.0,Queens,081,QN08,St. Albans,7.741275e+07,45401.316803,"POLYGON ((-73.75205 40.70523, -73.75174 40.704...",8108,8,St. Albans,86.5,85.3,157.1,225.7,752
1,2.0,Bronx,005,BX28,Van Cortlandt Village,2.566612e+07,21945.719299,"POLYGON ((-73.88705 40.88435, -73.88705 40.884...",528,8,Van Cortlandt Village,90.3,90.2,239.7,381.0,1108


In [23]:
nta_m_gdf = gpd.GeoDataFrame(nta_m, geometry='geometry')
nta_m_gdf = nta_m_gdf.set_crs(4326, allow_override=True)
nta_m_gdf.head(2)

,boro_code,boro_name,county_fip,ntacode,ntaname,shape_area,shape_leng,geometry,GeoID,GeoRank,Geography,aaa_rate,avg_rat_18,avg_rat_5,avg_rat_0,avg_num
0,4.0,Queens,081,QN08,St. Albans,7.741275e+07,45401.316803,"POLYGON ((-73.75205 40.70523, -73.75174 40.704...",8108,8,St. Albans,86.5,85.3,157.1,225.7,752
1,2.0,Bronx,005,BX28,Van Cortlandt Village,2.566612e+07,21945.719299,"POLYGON ((-73.88705 40.88435, -73.88705 40.884...",528,8,Van Cortlandt Village,90.3,90.2,239.7,381.0,1108


In [24]:
nta_m_gdf.to_file('../Data/asthma_er_nta/asthma_er_nta.shp')

C:\Users\mengk\anaconda3\envs\geo_test\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
